In [1]:
from ctgomartini.api import MartiniTopFile
from function import *

def CompareResults(working_dir, epsilon_r = 15):
    os.chdir(os.path.join(working_dir, "openmm"))
    strfile = "minimized.gro"
    topfile = "system.top"

    simulation = OMM_setSimulation(strfile, topfile, epsilon_r=epsilon_r, temperature=310.15, double_precision=True)
    OMM_calStrfile(strfile, simulation, set_vsite=True)

    omm_energy=Load_energy(clean=True)
    omm_forces=Load_forces(clean=True)
        
    # gmx
    os.chdir(os.path.join(working_dir, "gmx"))

    # Use the results from the last run
    # GMX_set(CreateMDP=False, double_precision=True)
    # GMX_run()

    gmx_energy=Load_energy(clean=False)
    gmx_forces=Load_forces(clean=False)   

    # Compare
    print("########################################")
    print(working_dir)
    result_energy, energy1, energy2, abs_energy_error, relative_energy_error=Compare_energy(omm_energy[:,1:], gmx_energy[:,1:], isPrint=True)
    result_forces, force1, force2, abs_force_error, relative_force_error=Compare_forces(omm_forces[:,1:], gmx_forces[:,1:], isPrint=True)
    if not (result_energy and result_forces):
        raise AssertionError("Energies or forces do not match.")
    
    print(energy1, energy2, abs_energy_error, relative_energy_error)
    print(force1, force2, abs_force_error, relative_force_error)
# def Compare_energy(energy1, energy2, isPrint=True):
#     energy1 = float(energy1)
#     energy2 = float(energy2)

#     relative_energy_error = abs(energy1 - energy2) * \
#         2 / (abs(energy1)+abs(energy2))
#     abs_energy_error = abs(energy1 - energy2)

#     if isPrint:
#         print('Energy Compare')
#         print(f'Absolute error: {abs_energy_error:.2e}')
#         print(f'Relative error: {relative_energy_error:.2e}')

#     if relative_energy_error <= 1e-5 or abs_energy_error <= 1e-3:
#         if isPrint:
#             print("Energies match!")
#         return True
#     else:
#         if isPrint:
#             print("Error: Energies do not match!")
#         return False
    
# def Compare_forces(forces1, forces2, isPrint=True):
#     average = 0.5*np.linalg.norm(forces1, axis=1) + 0.5*np.linalg.norm(forces2, axis=1)

#     relative_force_error = np.linalg.norm(forces1 - forces2, axis=1) / average
#     relative_force_error = np.nan_to_num(relative_force_error, nan=0)
#     max_relative_force_error, max_relative_force_error_index = relative_force_error.max(
#     ), relative_force_error.argmax()

#     abs_force_error = np.linalg.norm(forces1 - forces2, axis=1)
#     max_abs_force_error, max_abs_force_error_index = abs_force_error.max(
#     ), abs_force_error.argmax()

#     atol = 1e-5
#     rtol = 1e-4
#     # allclose = abs_force_error-(atol+rtol*average)
#     allclose = abs_force_error-(atol+rtol*np.linalg.norm(forces2, axis=1)) # use gmx forces as the standard
#     max_allclose = allclose.max()
#     if isPrint:
#         print('###Forces Compare###')
#         print(f'Max absolute error: {max_abs_force_error:.2e}')
#         print(f'Max relative error: {max_relative_force_error:.2e}')
#         print(f'      Max allclose: {max_allclose:.2e}')

#     if max_allclose <= 0:
#         if isPrint:
#             print("Forces match!")
#         return True
#     else:
#         if isPrint:
#             print("Error: Forces do not match!")
#         return False 

In [23]:
def Compare_energy(energy1, energy2, isPrint=True):
    energy1 = float(energy1)
    energy2 = float(energy2)

    relative_energy_error = abs(energy1 - energy2) * \
        2 / (abs(energy1)+abs(energy2))
    abs_energy_error = abs(energy1 - energy2)

    if isPrint:
        print('Energy Compare')
        print(f'Absolute error: {abs_energy_error:.2e}')
        print(f'Relative error: {relative_energy_error:.2e}')

    if relative_energy_error <= 1e-5 or abs_energy_error <= 1e-3:
        if isPrint:
            print("Energies match!")
        result = True
    else:
        if isPrint:
            print("Error: Energies do not match!")
        result = False
    
    return result, energy1, energy2, abs_energy_error, relative_energy_error


    
def Compare_forces(forces1, forces2, isPrint=True):
    average = 0.5*np.linalg.norm(forces1, axis=1) + 0.5*np.linalg.norm(forces2, axis=1)

    relative_force_error = np.linalg.norm(forces1 - forces2, axis=1) / average
    relative_force_error = np.nan_to_num(relative_force_error, nan=0)
    max_relative_force_error, max_relative_force_error_index = relative_force_error.max(
    ), relative_force_error.argmax()

    abs_force_error = np.linalg.norm(forces1 - forces2, axis=1)
    max_abs_force_error, max_abs_force_error_index = abs_force_error.max(
    ), abs_force_error.argmax()

    atol = 1e-5
    rtol = 1e-4
    # allclose = abs_force_error-(atol+rtol*average)
    allclose = abs_force_error-(atol+rtol*np.linalg.norm(forces2, axis=1)) # use gmx forces as the standard
    # Create a mask that excludes zeros
    mask = np.linalg.norm(forces2, axis=1) != 0

    max_allclose = allclose[mask].max()
    max_allclose_index = np.where(allclose == max_allclose)[0][0]
    max_abs_force_error = abs_force_error[max_allclose_index]
    max_relative_force_error = relative_force_error[max_allclose_index]
    if isPrint:
        print('###Forces Compare###')
        print(f'Max absolute error: {max_abs_force_error:.2e}')
        print(f'Max relative error: {max_relative_force_error:.2e}')
        print(f'      Max allclose: {max_allclose:.2e}')

    if max_allclose <= 0:
        if isPrint:
            print("Forces match!")
        result = True
    else:
        if isPrint:
            print("Error: Forces do not match!")
        result = False 

    return result, np.linalg.norm(forces1, axis=1)[max_allclose_index], np.linalg.norm(forces2, axis=1)[max_allclose_index],  max_abs_force_error, max_relative_force_error

In [19]:
# def Compare_forces(omm_forces, gmx_forces, isPrint=True):
#     f_atol = 1e-5
#     f_rtol = 1e-4

#     g_f = gmx_forces[:, :]
#     o_f = omm_forces[:, :]
#     errors = np.abs(omm_forces - gmx_forces) - f_atol - f_rtol * np.abs(gmx_forces)
#     print(errors.shape)
#     print(errors[:10,:])
#     max_ind = np.unravel_index(np.argmax(errors, axis=None), errors.shape)
#     print(np.max(errors, axis=None))
#     max_g = gmx_forces[max_ind[0], :]
#     max_o = omm_forces[max_ind[0], :]
#     max_d = np.abs(max_g - max_o)
#     max_e = errors[max_ind[0], :]
#     print()
#     print(f"    Largest violation is at: ({max_ind[0]}, {max_ind[1]}).")
#     print(f"    Gromacs: {max_g[0]:20f} {max_g[1]:20f} {max_g[2]:20f}")
#     print(f"     OpenMM: {max_o[0]:20f} {max_o[1]:20f} {max_o[2]:20f}")
#     print(f" Difference: {max_d[0]:20f} {max_d[1]:20f} {max_d[2]:20f}")
#     print(f" Difference: {max_e[0]:20f} {max_e[1]:20f} {max_e[2]:20f}")
#     print()

#     return True

In [3]:
# base_dir = os.getcwd()
base_dir = "/home/ys/CommonUse/Martini/CTGoMartini/tests/api"
print(base_dir)

/home/ys/CommonUse/Martini/CTGoMartini/tests/api


In [24]:
path = base_dir
# Martini 2 Protein-ENM, Protein-Go, Lipid, 

# Ubiquitin setup with Martini 2.2 force field
working_dir = os.path.join(path, "../data/NormalTopology/protein")
CompareResults(working_dir, epsilon_r = 15)  

working_dir = os.path.join(path, "../data/NormalTopology/beta-hairpin_go_m2")
CompareResults(working_dir, epsilon_r = 15)  

# 128 DPPC
working_dir = os.path.join(path, "../data/NormalTopology/simple_lipid")
CompareResults(working_dir, epsilon_r = 15)  

# Martini 2 Pol water
working_dir = os.path.join(path, "../data/NormalTopology/pol_water")
CompareResults(working_dir, epsilon_r = 2.5)  

# Martini 3 Protein-ENM, Protein-Go, Lipid 
working_dir = os.path.join(path, "../data/NormalTopology/1k6u_en_m3")
CompareResults(working_dir, epsilon_r = 15)  
working_dir = os.path.join(path, "../data/NormalTopology/1ubq_en_m3")
CompareResults(working_dir, epsilon_r = 15)  

working_dir = os.path.join(path, "../data/NormalTopology/1k6u_go_m3")
CompareResults(working_dir, epsilon_r = 15)  
working_dir = os.path.join(path, "../data/NormalTopology/1ubq_go_m3")
CompareResults(working_dir, epsilon_r = 15)
working_dir = os.path.join(path, "../data/NormalTopology/GlnBP_go_m3")
CompareResults(working_dir, epsilon_r = 15)   

working_dir = os.path.join(path, "../data/NormalTopology/popc_m3")
CompareResults(working_dir, epsilon_r = 15)  

# Aquaporin with elastic network, membrane protein
working_dir = os.path.join(path, "../data/NormalTopology/1j4n_en_m3")
CompareResults(working_dir, epsilon_r = 15)  


########################################
/home/ys/CommonUse/Martini/CTGoMartini/tests/api/../data/NormalTopology/protein
Energy Compare
Absolute error: 5.16e-07
Relative error: 2.14e-11
Energies match!
###Forces Compare###
Max absolute error: 5.71e-06
Max relative error: 8.74e-07
      Max allclose: -6.57e-04
Forces match!
-24144.3729675157 -24144.372967 5.157016857992858e-07 2.1359083812142403e-11
6.531592519988238 6.5315899581343135 5.711096432130817e-06 8.743805641076245e-07
########################################
/home/ys/CommonUse/Martini/CTGoMartini/tests/api/../data/NormalTopology/beta-hairpin_go_m2
Energy Compare
Absolute error: 5.91e-07
Relative error: 2.48e-09
Energies match!
###Forces Compare###
Max absolute error: 7.92e-05
Max relative error: 7.61e-07
      Max allclose: -1.03e-02
Forces match!
-238.37567459067228 -238.375674 5.906722719828394e-07 2.4779049864417986e-09
104.04902284774688 104.04907118047714 7.920911465388717e-05 7.612670843890013e-07
######################

/tmp/ipykernel_1267588/2189010770.py:30: RuntimeWarning: invalid value encountered in divide
  relative_force_error = np.linalg.norm(forces1 - forces2, axis=1) / average


########################################
/home/ys/CommonUse/Martini/CTGoMartini/tests/api/../data/NormalTopology/1ubq_go_m3
Energy Compare
Absolute error: 4.63e-06
Relative error: 1.96e-11
Energies match!
###Forces Compare###
Max absolute error: 1.67e-06
Max relative error: 1.11e-06
      Max allclose: -1.59e-04
Forces match!
-236273.47424562668 -236273.474241 4.62669413536787e-06 1.958194482129349e-11
1.5050966032265087 1.5050950183111365 1.6740759888503191e-06 1.1122720406717358e-06
########################################
/home/ys/CommonUse/Martini/CTGoMartini/tests/api/../data/NormalTopology/GlnBP_go_m3
Energy Compare
Absolute error: 1.21e-05
Relative error: 1.92e-11
Energies match!
###Forces Compare###
Max absolute error: 1.30e-06
Max relative error: 5.63e-07
      Max allclose: -2.39e-04
Forces match!
-630565.7566500785 -630565.756638 1.2078438885509968e-05 1.915492358765744e-11
2.3013096923324388 2.301309325272217 1.2957907524185843e-06 5.630667007033545e-07
####################